In [94]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO
import matplotlib.image as mpimg
from sklearn import tree
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [95]:
USD=1
EUR=1.13566
GLD=1.22477
RUR=65.6664

In [96]:
path1="H:\LtsDoModel!\RawData.csv"

df1 = pd.read_csv(path1, sep=";")
print("Done")

Done


In [97]:
path2="H:\LtsDoModel!\ClientsInfo.csv"

df2 = pd.read_csv(path2, sep=";")
print("Done")

Done


In [98]:
df=df1.merge(df2, how='inner', on='Login')

In [99]:
df[['Login', 'Leverage', 'MarketExecution', 'client_id', 'agent_id']] = df[['Login', 'Leverage', 'MarketExecution', 'client_id', 'agent_id']].astype('object')

In [100]:
df["Profit Pips"].replace(np.nan, 0, inplace = True)
df["Loss Pips"].replace(np.nan, 0, inplace = True)
df["agent_id"].replace(np.nan, 0, inplace = True)

In [101]:
df["agent_id"]=df["agent_id"].astype(bool)
df["agent_id"]=df["agent_id"].astype(int)

In [102]:
df.loc[df['Currency']=='RUR', ['Fin.Result Fx Majors','Fin.Result Fx Minors','Fin.Result Fx Exotics',
'Fin.Result Fx Special','Fin.Result Spot Metals', 'Fin.Result Fx Rub', 'Fin.Result Crypto',
'Fin.Result CFD', 'Fin.Result', 'FastDeals Fin.Result','MaxBalance', 'MinFloating', 'Equity','Future Clearing']] /= RUR

In [103]:
df.loc[df['Currency']=='EUR', ['Fin.Result Fx Majors','Fin.Result Fx Minors','Fin.Result Fx Exotics',
'Fin.Result Fx Special','Fin.Result Spot Metals', 'Fin.Result Fx Rub', 'Fin.Result Crypto',
'Fin.Result CFD', 'Fin.Result', 'FastDeals Fin.Result','MaxBalance', 'MinFloating', 'Equity','Future Clearing']] *= EUR

In [104]:
df.loc[df['Currency']=='GLD', ['Fin.Result Fx Majors','Fin.Result Fx Minors','Fin.Result Fx Exotics',
'Fin.Result Fx Special','Fin.Result Spot Metals', 'Fin.Result Fx Rub', 'Fin.Result Crypto',
'Fin.Result CFD', 'Fin.Result', 'FastDeals Fin.Result','MaxBalance', 'MinFloating', 'Equity','Future Clearing']] *= GLD

In [105]:
df.describe(include='all')

,Login,MarketExecution,AccountType,Country,Currency,Leverage,Fin.Result Fx Majors,Fin.Result Fx Minors,Fin.Result Fx Exotics,Fin.Result Fx Special,...,Loss Pips,Min MarginLevel,MaxBalance,MinFloating,Equity,Turnover USD,Future Clearing,client_id,agent_id,Client Name
count,35446.0,35446.0,35446,35446,35446,35446.0,35446.000000,35446.000000,35446.000000,35446.000000,...,3.544600e+04,35446.000000,3.544600e+04,35446.000000,35446.000000,3.544600e+04,35446.000000,35446.0,35446.000000,35446
unique,35446.0,2.0,3,154,4,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31164.0,NaN,29745
top,5322718.0,0.0,Standard,Russian Federation,USD,500.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10921118.0,NaN,Sergey Sannikov
freq,1.0,22240.0,22240,22500,25535,26695.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,NaN,43
mean,NaN,NaN,NaN,NaN,NaN,NaN,0.136046,-5.764008,-2.970074,-0.037437,...,-8.070458e+03,1217.889005,3.149210e+03,-919.396326,2028.379629,3.802914e+06,-223.282216,NaN,0.294786,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,3804.315442,349.171937,202.408349,4.909746,...,5.170472e+04,6488.740977,2.413783e+04,10878.230475,17477.062891,2.579850e+07,5409.643304,NaN,0.455953,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,-97619.920000,-19413.120000,-14260.930000,-805.660000,...,-3.024462e+06,0.000000,-2.011580e+03,-690519.330000,-4590.180000,1.330000e+00,-586264.490000,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,-76.394386,0.000000,0.000000,0.000000,...,0.000000e+00,104.010000,7.438500e+01,-158.020000,6.490000,6.065100e+04,-39.067500,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,-4.984817,0.000000,0.000000,0.000000,...,0.000000e+00,189.145000,2.771037e+02,-22.580000,97.227710,2.886262e+05,0.000000,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,27.330000,0.000000,0.000000,0.000000,...,0.000000e+00,777.835000,1.045650e+03,-1.730141,535.082500,1.339068e+06,10.328075,NaN,1.000000,NaN


In [106]:
df.drop(labels='Currency', axis=1, inplace=True)

In [107]:
#df.set_index('Login', inplace=True)

In [108]:
df[['Leverage', 'MarketExecution', 'Fin.Result', 'FastDeals Fin.Result','MaxBalance', 'MinFloating', 'Equity', 'Profit Pips', 'Loss Pips', 'Min MarginLevel','Future Clearing']].corr()

,Fin.Result,FastDeals Fin.Result,MaxBalance,MinFloating,Equity,Profit Pips,Loss Pips,Min MarginLevel,Future Clearing
Fin.Result,1.000000,0.266178,0.300064,-0.126092,0.349208,0.037018,0.049981,0.004179,0.005232
FastDeals Fin.Result,0.266178,1.000000,0.053026,-0.051758,0.068412,-0.010948,0.018957,-0.002036,-0.121098
MaxBalance,0.300064,0.053026,1.000000,-0.743901,0.913816,0.124063,-0.058592,0.022524,-0.214605
MinFloating,-0.126092,-0.051758,-0.743901,1.000000,-0.443875,-0.050089,0.039166,0.000099,0.314164
Equity,0.349208,0.068412,0.913816,-0.443875,1.000000,0.149720,-0.055046,0.031701,-0.086003
Profit Pips,0.037018,-0.010948,0.124063,-0.050089,0.149720,1.000000,-0.552724,-0.014211,-0.004313
Loss Pips,0.049981,0.018957,-0.058592,0.039166,-0.055046,-0.552724,1.000000,0.020191,-0.000956
Min MarginLevel,0.004179,-0.002036,0.022524,0.000099,0.031701,-0.014211,0.020191,1.000000,0.003236
Future Clearing,0.005232,-0.121098,-0.214605,0.314164,-0.086003,-0.004313,-0.000956,0.003236,1.000000


In [109]:
df.columns

Index(['Login', 'MarketExecution', 'AccountType', 'Country', 'Leverage',
       'Fin.Result Fx Majors', 'Fin.Result Fx Minors', 'Fin.Result Fx Exotics',
       'Fin.Result Fx Special', 'Fin.Result Spot Metals', 'Fin.Result Fx Rub',
       'Fin.Result Crypto', 'Fin.Result CFD', 'Fin.Result',
       'FastDeals Fin.Result', 'Loss Deals Count', 'Profit Deals Count',
       'Profit Pips', 'Loss Pips', 'Min MarginLevel', 'MaxBalance',
       'MinFloating', 'Equity', 'Turnover USD', 'Future Clearing', 'client_id',
       'agent_id', 'Client Name'],
      dtype='object')

In [110]:
bins = pd.IntervalIndex.from_tuples([(-9999999, -5000), (-5000, 5000), (5000, 9999999)])
df['Future Clearing'] = pd.cut(df['Future Clearing'], bins)

In [111]:
Y=df['Future Clearing'].cat.codes

In [112]:
Y = Y.replace(to_replace={0 : 'Client will lose', 
                                  1 : 'Client will near zero', 
                                  2:'Client will make profit'} )

In [113]:
X = df.drop(labels=['Login', 'Client Name', 'client_id', 'AccountType', 'Country', 'Future Clearing'], axis=1)

In [139]:
newdf= df.drop(labels=['Login', 'Client Name', 'client_id', 'AccountType', 'Country', 'Future Clearing'], axis=1)

In [114]:
X=X.values

In [124]:
df['Leverage'].unique()

array([500.0, 1000.0, 200.0, 100.0, 50.0, 25.0, 10.0, 5.0, 1.0, 33.0],
      dtype=object)

In [131]:
X[:,1]

array([500.0, 500.0, 1000.0, ..., 500.0, 100.0, 1000.0], dtype=object)

In [135]:
from sklearn import preprocessing
dummy_Leverage=preprocessing.LabelEncoder()
dummy_Leverage.fit([500.0, 1000.0, 200.0, 100.0, 50.0, 25.0, 10.0, 5.0, 1.0, 33.0])
X[:,1]=dummy_Leverage.transform(X[:,1])

In [141]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, Y, test_size=0.3, random_state=3)

In [142]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [143]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [145]:
print (predTree [0:5])
print (y_testset [0:5])

['Client will near zero' 'Client will near zero' 'Client will near zero'
 'Client will near zero' 'Client will near zero']
31307    Client will near zero
27064    Client will near zero
32081    Client will near zero
6660     Client will near zero
32864    Client will near zero
dtype: object


In [150]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_testset, predTree))

DecisionTrees's Accuracy:  0.9871167951852549


In [157]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [159]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = newdf.columns
targetNames = df['Future Clearing'].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

InvocationException: GraphViz's executables not found